In [1]:
import requests
import pandas as pd
from tqdm import tqdm
import time

In [2]:

API_KEY = "1b0fb19cd3fa40d5aa716747d08eee46"  
BASE_URL = "https://api.rawg.io/api/games"

# Danh sách lưu slug của 200 game đầu tiên
game_slugs = []

# Lấy 200 game phổ biến đầu tiên (10 trang, mỗi trang 20 game)
for page in range(1, 11):
    params = {
        "key": API_KEY,
        "page": page,
        "page_size": 20
    }
    res = requests.get(BASE_URL, params=params)
    data = res.json()
    for game in data["results"]:
        game_slugs.append(game["slug"])

# Lấy thông tin chi tiết cho từng game
game_details = []

for slug in tqdm(game_slugs, desc="Fetching game details"):
    detail_url = f"{BASE_URL}/{slug}"
    params = {"key": API_KEY}
    res = requests.get(detail_url, params=params)
    if res.status_code != 200:
        print(f"Failed to fetch {slug}")
        continue

    g = res.json()

    # Parse dữ liệu mong muốn
    game_details.append({
        "name": g.get("name"),
        "description_raw": g.get("description_raw"),
        "metacritic": g.get("metacritic"),
        "playtime": g.get("playtime"),
        "tags": [tag["name"] for tag in g.get("tags", [])],
        "stores": [store["store"]["name"] for store in g.get("stores", [])],
        "esrb_rating": g["esrb_rating"]["name"] if g.get("esrb_rating") else None,
        "developers": [dev["name"] for dev in g.get("developers", [])],
        "publishers": [pub["name"] for pub in g.get("publishers", [])],
        "updated": g.get("updated"),
        "added_by_status": g.get("added_by_status")
    })

    time.sleep(0.3)  # Tránh gọi API quá nhanh

# Lưu vào file CSV để phân tích sau
df = pd.DataFrame(game_details)
df.to_csv("rawg_200_games_details.csv", index=False)

print("Done. Data saved to rawg_200_games_details.csv")


Fetching game details: 100%|██████████| 200/200 [03:21<00:00,  1.01s/it]

Done. Data saved to rawg_200_games_details.csv


In [7]:
df = pd.read_csv("rawg_200_games_details.csv")
df.head()

,name,description_raw,metacritic,playtime,tags,stores,esrb_rating,developers,publishers,updated,added_by_status
0,Grand Theft Auto V,"Rockstar Games went bigger, since their previo...",92.0,74,"['Singleplayer', 'Steam Achievements', 'Multip...","['Steam', 'PlayStation Store', 'Epic Games', '...",Mature,"['Rockstar North', 'Rockstar Games']",['Rockstar Games'],2025-05-26T14:58:48,"{'yet': 557, 'owned': 12702, 'beaten': 6240, '..."
1,The Witcher 3: Wild Hunt,"The third game in a series, it holds nothing b...",92.0,43,"['Singleplayer', 'Full controller support', 'A...","['GOG', 'PlayStation Store', 'Steam', 'Xbox St...",Mature,['CD PROJEKT RED'],['CD PROJEKT RED'],2025-05-25T18:05:53,"{'yet': 1173, 'owned': 12701, 'beaten': 5083, ..."
2,Portal 2,Portal 2 is a first-person puzzle game develop...,95.0,11,"['Singleplayer', 'Steam Achievements', 'Multip...","['Xbox Store', 'Steam', 'PlayStation Store', '...",Everyone 10+,['Valve Software'],"['Electronic Arts', 'Valve']",2025-05-26T14:58:22,"{'yet': 657, 'owned': 13004, 'beaten': 5711, '..."
3,Counter-Strike: Global Offensive,Counter-Strike is a multiplayer phenomenon in ...,81.0,64,"['Steam Achievements', 'Multiplayer', 'Full co...","['PlayStation Store', 'Steam', 'Xbox 360 Store']",Mature,"['Valve Software', 'Hidden Path Entertainment']",['Valve'],2025-05-26T16:11:35,"{'yet': 268, 'owned': 14083, 'beaten': 1105, '..."
4,Tomb Raider (2013),A cinematic revival of the series in its actio...,86.0,10,"['Singleplayer', 'Multiplayer', 'Full controll...","['Xbox 360 Store', 'Steam', 'PlayStation Store...",Mature,['Crystal Dynamics'],['Square Enix'],2025-05-23T20:02:39,"{'yet': 695, 'owned': 11542, 'beaten': 4393, '..."


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             200 non-null    object 
 1   description_raw  200 non-null    object 
 2   metacritic       187 non-null    float64
 3   playtime         200 non-null    int64  
 4   tags             200 non-null    object 
 5   stores           200 non-null    object 
 6   esrb_rating      165 non-null    object 
 7   developers       200 non-null    object 
 8   publishers       200 non-null    object 
 9   updated          200 non-null    object 
 10  added_by_status  200 non-null    object 
dtypes: float64(1), int64(1), object(9)
memory usage: 17.3+ KB


In [4]:

API_KEY = "1b0fb19cd3fa40d5aa716747d08eee46" 
BASE_URL = "https://api.rawg.io/api/games"

# Số lượng game bạn muốn lấy (ví dụ: 1000)
TOTAL_GAMES = 1000
PAGE_SIZE = 40  # Tối đa mà RAWG cho phép là 40 game mỗi trang
total_pages = (TOTAL_GAMES // PAGE_SIZE) + 1

game_slugs = []

# B1: Lấy slug cho tất cả game
for page in tqdm(range(1, total_pages + 1), desc="Fetching game list"):
    params = {
        "key": API_KEY,
        "page": page,
        "page_size": PAGE_SIZE
    }
    res = requests.get(BASE_URL, params=params)
    if res.status_code != 200:
        print(f"⚠️ Failed to get page {page}")
        continue

    data = res.json()
    for game in data.get("results", []):
        game_slugs.append(game["slug"])

    time.sleep(0.2)  # nghỉ chút để tránh bị giới hạn tốc độ

print(f"✅ Collected {len(game_slugs)} game slugs")

# B2: Lấy chi tiết cho từng game
game_details = []

for slug in tqdm(game_slugs, desc="Fetching game details"):
    detail_url = f"{BASE_URL}/{slug}"
    res = requests.get(detail_url, params={"key": API_KEY})
    if res.status_code != 200:
        print(f"Failed to fetch details for {slug}")
        continue

    g = res.json()

    game_details.append({
        "name": g.get("name"),
        "description_raw": g.get("description_raw"),
        "metacritic": g.get("metacritic"),
        "playtime": g.get("playtime"),
        "tags": [tag["name"] for tag in g.get("tags", [])],
        "stores": [store["store"]["name"] for store in g.get("stores", [])],
        "esrb_rating": g["esrb_rating"]["name"] if g.get("esrb_rating") else None,
        "developers": [dev["name"] for dev in g.get("developers", [])],
        "publishers": [pub["name"] for pub in g.get("publishers", [])],
        "updated": g.get("updated"),
        "added_by_status": g.get("added_by_status")
    })

    time.sleep(0.3)

# B3: Lưu kết quả ra CSV
df = pd.DataFrame(game_details)
df.to_csv("rawg_games_1000.csv", index=False)

print("Done! Saved to rawg_games_1000.csv")


Fetching game list: 100%|██████████| 26/26 [00:48<00:00,  1.87s/it]


✅ Collected 1040 game slugs


Fetching game details:  99%|█████████▉| 1033/1040 [17:07<00:10,  1.50s/it]

Failed to fetch details for owlboy


Fetching game details:  99%|█████████▉| 1034/1040 [17:07<00:07,  1.17s/it]

...

Failed to fetch details for a-plague-tale-requiem


Fetching game details: 100%|█████████▉| 1038/1040 [17:10<00:01,  1.39it/s]

Failed to fetch details for project-cars-2


Fetching game details: 100%|██████████| 1040/1040 [17:11<00:00,  1.01it/s]

Failed to fetch details for blacklight-retribution
Done! Saved to rawg_games_1000.csv
